## Анализ результатов опроса с использованием Radar Chart

Дано:
1. CSV-файл с результатами опроса по различным критериям.
2. URL для загрузки данных из CSV-файла.
3. Константное значение 2, которое будет отображаться второй линией на графике.

Необходимо:
1. Выполнить выборку данных опроса из колонок с 6 по 16.
2. Построить Radar Chart с использованием библиотеки Plotly.
3. Использовать результаты опроса

TODO:
1. Проверить датафреймы. Похоже перемудрил с количеством
2. По хорошему - продумать модель данных - инфо отдельно, скилы - отдельно    

### Решение

1. Импорт библиотек

In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

2. Подготовка

Заполняем настроечные данные:
   - URL для загрузки
   - референсное значение
   - уровни компетенций

In [2]:
url = 'https://docs.google.com/spreadsheets/d/1bRtJkFXv3DfOjl19jlkJee65u_BPR1e6VLdGTIM8Gcc/export?format=csv'

reference = 2

expertiseLvl = [
    'Не знаю вообще как',
    'Знаю только теорию',
    'Знаю теорию и применяю на практике, но есть сложности',
    'Использую на практике, теории не знаю',
    'Владею в совершенстве и активно использую на практике'
]

Создаем пустые датафреймы:
   - рабочий
   - для учета опыта

In [3]:
data = {'Category': [],
        'Value': []}

experience = {'Kind': [],
              'Level': []}

Читаем csv в датафрем для обработки

In [4]:
df = pd.read_csv(url,header=[0,1])

df = df.dropna() # исключаем из обработки незавершенные строки

Определяем количество столбцов - пригодится в дальнейшем для итерации в циклах

In [5]:
numOfCol = len(df.columns)

3. Обработка

Готовим справочную информацию для легенды:

- Заголовок (находится в строке 1 столбца 7)

In [6]:
expertise = str(df.columns[6][0]).rstrip(". ")

- Список компетенций (в строке 2 столбцов 7 - 16)

In [7]:
expertiseKinds = []

for i in range(10):
    spacePos = df.columns[6+i][1].find(' ')             # находим пробел после номера компетенции
    exprtKind = str(df.columns[6+i][1])[spacePos+1:-1]  # обрезаем номер у компетенции
    expertiseKinds.append(exprtKind)                    # добавляем полученное значение в список

Добавляем категории в рабочий датафрейм:
- проходим в цикле по столбцам 1 - 6
- дополнительно добавляем категорию для опыта

In [8]:
for i in range(7):
    if i==3:
        addItem = 'level'       # представленное в файле не устраивает - вводим свое имя категории
    elif i==4:
        addItem = 'available'   # представленное в файле не устраивает - вводим свое имя категории
    elif i==5:
        addItem = 'work type'   # представленное в файле не устраивает - вводим свое имя категории
    elif i==6:
        addItem = 'ExpLvl'      # дополнительная категория для опыта
    else:                       # остальные имена устраивают - берем как есть из загруженного файла
        addItem = df.columns[i][1]    
    data['Category'].append(addItem)

В цикле по оставшимся столбцам (7-16) добавляем категории в датафрейм опыта

<details>
	<summary>Информация для добавления</summary>
		Номер компетенции (в пределах группы)
</details>

In [9]:
for i in range(6,numOfCol):    
    spacePos = df.columns[i][1].find(' ')   # находим пробел после номера компетенции
    dotPos = df.columns[i][1].find('.')     # находим '.' в номере компетенции
    addItem = df.columns[i][1][2:spacePos]  # берем все, что между '.' и пробелом
    experience['Kind'].append(addItem)

Задаем образец для сравнения - добавляем в датафрейм опыта референсные значения

In [10]:
for i in range(6,numOfCol):
    experience['Level'].append(reference)

Добавляем референсные значения в рабочий датафрейм

<details>
	<summary>Информация для добавления</summary>
    
    name:       Reference
    phone:      -
    email:      -
    level:      -
    available:  -
    work type:  -
    ExpLvl:     2
</details>

In [11]:
row = []
for i in range(5):
    if i==0:
        row.append('Reference')
    row.append('-')
row.append(experience['Level'])
data['Value'].append(row)

Добавляем в рабочий датафрейм значения из каждой строки файла данных, параллельно выполняем форматирование и обработку

In [12]:
for i in df.index:
    row = []
    expRow = []
    # Заполняем информационные поля
    for j in range(6):
        if j==1:
            # фоматируем номер телефона к удобочитаемому формату
            try:
                s = str(int(df.values[i][j]))
                addItem = '+{} ({}) {}-{}-{}'.format(s[0], s[1:4], s[4:7], s[7:9], s[9:11])
            except:
                addItem = '-' # если не получается - добавляем '-'
                continue
        else:
            # значения информационных полей
            try:
                addItem = df.values[i][j]
            except:
                addItem = '-' # если не получается - добавляем '-'
                continue
        row.append(addItem)
    # Заполняем поля с уровнями знаний
    for k in range(6,numOfCol):
        try:
            addItem = int(df.values[i][k][:1])
        except:
            addItem = '-' # если не получается - добавляем '-'
            continue
        expRow.append(addItem)
    row.append(expRow)
    data['Value'].append(row)

4. Подготовка отчета для отображения информации

Готовим холст:
- Нарезаем раскладку блоков информации (3 строки 1 столбец)
- Добавляем названия блоков
- Настраиваем высоту блоков (соотношение) и расстояние между ними

In [13]:
fig = make_subplots(rows=3, cols=1,
                    specs=[[{'type':'table'}],
                           [{'type':'polar'}],
                           [{'type':'table'}]],
                    row_heights=[3,6,7],
                    subplot_titles=('Инфо','','Список компетенций'),
                    vertical_spacing = 0.05
)

Добавляем данные в первый блок - таблица информации

Данные берем из рабочего датафрейма 

In [ ]:
fig.add_trace(go.Table(
    hoverinfo=None,
    columnwidth=[0.1,0.9,0],
    header=dict(align='left',
               height=0,
               values=['','']),
    cells=dict(align='left',
               height=25,
               values=[data['Category'][:6],data['Value'][1][:6]])
),
row=1, col=1
)

Добавляем данные во второй блок - RadarChart

Подготавливаем угловые координаты

In [15]:
theta = experience['Kind'].copy()
theta.append(experience['Kind'][0])

Для каждой строки из рабочего датафрейма готовим трейс

In [16]:
for i in range(len(data)):
    
    # Подготавливаем радиальные координаты
    values = data['Value'][i][6].copy()
    values.append(data['Value'][i][6][0])
    
    # Готовим customdata для hover
    custData=[]
    for v in values:
        custData.append(expertiseLvl[v-1])
    
    # Задаем цвета для маркеров:
    # все, что выше или равно референсному значению - зеленое
    # все, что ниже референсного значения - красное
    colors = ['rgba(72,161,56,1)' if int(v)>=reference else 'rgba(226,120,123,1)' for v in values]
    
    # Рисуем график, даем ему имя, настраиваем размер точек, цвет линий, заливку, добавляем hover
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=theta,
        name=data['Value'][i][0],
        marker=dict(
            color=colors,
        size=8),
        line=dict(
            color='grey'
        ),
        fillcolor='rgba(226,120,123,0.2)' if i==0 else 'rgba(72,161,56,0.4)',
        mode='lines+markers',
        fill='tonext',
        customdata=custData,
        hovertemplate='%{customdata}'
    ),
    row=2, col=1)

Добавляем данные в третий блок - таблица компетенций

Данные берем из датафрейма опыта и датафрейма компетенций

In [ ]:
fig.add_trace(go.Table(
    hoverinfo=None,
    columnwidth=[0.06,0.86,0.08],
    header=dict(align='center',
        values=['Номер','Описание','Уровень владения']),
    cells=dict(align='left',
               height=25,
        values=[experience['Kind'],expertiseKinds,data['Value'][1][6]])
),
row=3, col=1
)

5. Настройка отчета для отображения информации

Заголовки блоков прижимаем к левому краю

In [18]:
for annotation in fig['layout']['annotations']:
    annotation['xanchor']='left'
    annotation['x']=0
    annotation['yref']='paper'

Настраиваем вывод:
- назначаем шаблон отрисовки
- добавляем название
- устанавливаем размеры
- выключаем отображение легенды
- для RadarChart рисуем размерную сетку

In [ ]:
fig.update_layout(
    template='ggplot2',
    title=dict(text=expertise),
    height=1200,
    width=1000,
    showlegend=False,    
    polar=dict(
        radialaxis=dict(
            range=[0,6],
            visible=True),
    ),
)

6. Выводим сформированный отчет

In [ ]:
fig.show()